In [1]:
from google.colab import drive
import os, sys
if 'Linux' in os.uname():
    from google.colab import drive
    drive.mount('/ME')
    predir='/ME/My Drive/'
else:
    predir='/Users/amit/Google Drive/'
datadirs=predir+'Colab Notebooks/FA/'
datadirsa=predir+'Colab Notebooks/STVAE/_CODE'

sys.path.insert(1, datadirs)
sys.path.insert(1, datadirsa)
savepath = datadirs+'save/'
datapath = predir+'LSDA_data'

Drive already mounted at /ME; to attempt to forcibly remount, call drive.mount("/ME", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from pprint import pprint
# layers
from layers import FALinear, FAConv2d, UfLinear, UfConv2d, UsLinear, UsConv2d
# Module classes and training and testing routines
from utils import *
from networks import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
dtype = torch.float32
# data
data = get_cifar10_man(datapath, num_train=45000)

cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [3]:
class pars:
    def __init__(self):

      self.OPT='SGD'
      self.layerwise=True
      self.device=device
      self.fa=True
      self.START_LAYER = 0
      self.NUM_LAYERS = 4
      self.NUM_CHANNEL = 32 
      self.HW = 32
      self.epochs = 200
      self.batch_size= 500
      self.MX=1.
      self.LR=[.1,.05,.025,.0125]
      self.CR='CE'
      self.get_net=get_net_c

pars=pars()
class layer_pars:
    def __init__(self,pars):
      self.NCOLD=3
      self.NC=pars.NUM_CHANNEL
      self.HW=pars.HW
pars.layer_pars=layer_pars(pars)


In [ ]:
logi=[[False, False],[True, False], [True, True]]
cori=['CE','H']

for ll in logi:
  for cc in cori:
    pars.layerwise=ll[0]
    pars.fa=ll[1]
    pars.CR=cc
    pars.layer_pars=layer_pars(pars)
    get_net_pre(pars)
    run_net(data,pars,savepath)
    show_results(pars, savepath)

{'CR': 'CE',
 'HW': 32,
 'LR': [0.1, 0.05, 0.025, 0.0125],
 'MX': 1.0,
 'NUM_CHANNEL': 32,
 'NUM_LAYERS': 4,
 'OPT': 'SGD',
 'START_LAYER': 0,
 'batch_size': 500,
 'criterion': CrossEntropyLoss(),
 'device': device(type='cuda', index=0),
 'epochs': 200,
 'fa': False,
 'get_net': <function get_net_c at 0x7fb94d9d2e18>,
 'layer_pars': <__main__.layer_pars object at 0x7fb9458bb4e0>,
 'layerwise': False}



LAYER:3
Sequential(
  (layer0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Hardtanh(min_val=0, max_val=1.0)
    (2): Dropout(p=0.5, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Hardtanh(min_val=0, max_val=1.0)
    (2): Dropout(p=0.5, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv